In [16]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import numpy as np
import autoreject
import csv
import matplotlib.pyplot as plt
import pandas as pd
import mne
import os
import sys
from atpbar import atpbar
import glob


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# Suppress MNE output
#mne.set_log_level('WARNING')
mne.set_log_level('ERROR')
#mne.set_log_level('CRITICAL')
#mne.set_log_level('INFO')



## Plot the GLOBAL evoked response to determine the time window I wish to investigate

In [18]:
#pick = ['Fz']
pick = ['Fpz', 'Fz', 'Cz', 'AF3', 'AF4', 'F3', 'F4', 'FC1', 'FC2']
#pick = ['Fpz', 'Fz', 'Cz', 'Pz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'F7', 'F8', 'F3', 'F4', 'FC1', 'FC2']

paths = glob.glob('/Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/MMN/epochs/*.fif')
epochs_list = [mne.read_epochs(path) for path in paths]
epochs_list_picks = []
for epoch in epochs_list:
    if not set(epoch.info['bads']).intersection(pick):
        epochs_list_picks.append(epoch.pick_channels(pick))
concatenated_epochs = mne.concatenate_epochs(epochs_list_picks)
evoked_global = concatenated_epochs.average()
fig = evoked_global.plot()

"""
0.11 to 0.13 --> [54:58]
0.11 to 0.19 --> [54:74]

0.21 s = 53.76 sample
0.23 s = 58.88 sample
0.29 s = 74.24 sample 
"""

'\n0.11 to 0.13 --> [54:58]\n0.11 to 0.19 --> [54:74]\n\n0.21 s = 53.76 sample\n0.23 s = 58.88 sample\n0.29 s = 74.24 sample \n'

## Make plots per subject & per condition, with resolution cpaturing all four conditions + extract values

In [27]:
pick = ['Fz', 'Cz', 'Fpz', 'F3', 'F4', 'AF3', 'AF4', 'FC1', 'FC2']
#pick = ['Fz']
show_plots = True

data = []
for sub in [0,1,2,3,4,5,6,7,8,9,'?']:
#for sub in [0,1]:
    paths_jhana = glob.glob(f'/Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/MMN/epochs/sub{sub}-day?-jhana_epo.fif')
    paths_mindf = glob.glob(f'/Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/MMN/epochs/sub{sub}-day?-mindfulness_epo.fif')
    
    epochs_list_jhana = [mne.read_epochs(path) for path in paths_jhana]
    epochs_list_mindf = [mne.read_epochs(path) for path in paths_mindf]
    
    epochs_list_picks_jhana = []
    epochs_list_picks_jhana_days = []
    for i,epoch in enumerate(epochs_list_jhana): 
        if not set(epoch.info['bads']).intersection(pick):
            epochs_list_picks_jhana.append(epoch.pick_channels(pick))
            epochs_list_picks_jhana_days.append(os.path.basename(paths_jhana[i])[5:9])
            
    
    epochs_list_picks_mindf = []
    epochs_list_picks_mindf_days = []
    for i, epoch in enumerate(epochs_list_mindf): 
        if not set(epoch.info['bads']).intersection(pick):
            epochs_list_picks_mindf.append(epoch.pick_channels(pick))
            epochs_list_picks_mindf_days.append(os.path.basename(paths_mindf[i])[5:9])

    if (len(epochs_list_picks_jhana)>1 and len(epochs_list_picks_mindf)>1):

        if show_plots: 
            concatenated_epochs_jhana = mne.concatenate_epochs(epochs_list_picks_jhana)
            concatenated_epochs_mindf = mne.concatenate_epochs(epochs_list_picks_mindf)
            
            evoked_std_mindf = concatenated_epochs_mindf['1'].average()
            evoked_odd_mindf = concatenated_epochs_mindf['-1'].average()
            evoked_std_jhana = concatenated_epochs_jhana['1'].average()
            evoked_odd_jhana = concatenated_epochs_jhana['-1'].average()
            
            times = evoked_std_mindf.times
                    
            std_mindf_avg = evoked_std_mindf.get_data().mean(axis=0)
            odd_mindf_avg = evoked_odd_mindf.get_data().mean(axis=0)
            mmn_mind = odd_mindf_avg - std_mindf_avg
            std_jhana_avg = evoked_std_jhana.get_data().mean(axis=0)
            odd_jhana_avg = evoked_odd_jhana.get_data().mean(axis=0)
            mmn_jhana = odd_jhana_avg - std_jhana_avg
            
            plt.figure(figsize=(10, 6))
            plt.plot(times, mmn_mind, label='mmn_mind', color='blue')
            plt.plot(times, mmn_jhana, label='mmn_jhana', color='green')
                        
            plt.xlabel('Time (s)')
            plt.ylabel('Amplitude (µV)')
            sample_n = len(concatenated_epochs_mindf)+len(concatenated_epochs_jhana)
            title = f'Averaged Evoked Responses Across All Channels for subject: {sub}, sample_n={sample_n}'
            plt.title(title)
            plt.axvline(0, color='black', linestyle='--', label='Stimulus Onset')  # Add a vertical line at stimulus onset
            plt.legend()
            
            #plt.ylim([-0.00001, 0.00001])
            
            plt.savefig(f'figures/{title}.png')

            #plt.show()
            
    
            if sub == '?':
                evoked_std_mindf = concatenated_epochs_mindf['1'].average()
                evoked_odd_mindf = concatenated_epochs_mindf['-1'].average()
                evoked_std_jhana = concatenated_epochs_jhana['1'].average()
                evoked_odd_jhana = concatenated_epochs_jhana['-1'].average()
                
                fig = evoked_std_mindf.plot()
                fig = evoked_odd_mindf.plot()
                fig = evoked_std_jhana.plot()
                fig = evoked_odd_jhana.plot()


    ################################
    # extract means into dataframe #
    ################################
    if sub != '?':
        for epoch_list, epoch_list_days, meditation_type in zip(
            [epochs_list_picks_jhana, epochs_list_picks_mindf],
            [epochs_list_picks_jhana_days, epochs_list_picks_mindf_days],
            ['jhana', 'mindfulness']
        ):
            for i, epochs in enumerate(epoch_list):
                day = epoch_list_days[i]
                for time_start, time_end, time_name in zip(
                    [54, 54],
                    [58, 74], 
                    ['early_short', 'early_long']
                    ):         
                    
                    mean_std = np.mean(epochs['1'].get_data()[:,:,time_start:time_end])*10**6
                    mean_odd = np.mean(epochs['-1'].get_data()[:,:,time_start:time_end])*10**6
                    mmn = mean_std - mean_odd
                    sample_n = len(epochs)
                    data.append([sub, day, meditation_type, time_name, pick, sample_n, mean_std, mean_odd, mmn])
                        
df = pd.DataFrame(data, columns=['sub', 'day', 'meditation_type', 'time_name', 'pick', 'sample_n', 'mean_std', 'mean_odd', 'mmn'])    
df.to_csv('mean_values.csv', index=False)


In [22]:
len(odd_mindf_avg)

232

In [9]:
plt.close('all')

In [10]:
concatenated_epochs_mindf

Number of events,7395
Events,-1: 11551: 6240
Time range,-0.102 – 0.801 s
Baseline,-0.100 – 0.000 s


In [11]:
concatenated_epochs_jhana

Number of events,8252
Events,-1: 13161: 6936
Time range,-0.102 – 0.801 s
Baseline,-0.100 – 0.000 s


In [13]:
1316/6936

0.18973471741637832

In [14]:
1155/6240

0.18509615384615385

## multi-level linear model 

In [47]:
import pandas as pd
import statsmodels.formula.api as smf

# Ensure categorical columns are treated as such
df['sub'] = df['sub'].astype('category')
df['day'] = df['day'].astype('category')
df['meditation_type'] = df['meditation_type'].astype('category')
df['time_name'] = df['time_name'].astype('category')

# Fit a mixed effects linear model with random intercepts for 'sub'
model = smf.mixedlm(
    "mmn ~ day + meditation_type + condition + time_name",  # Fixed effects
    df, 
    groups=df["sub"],  # Random effect for 'sub'
)

# Fit the model
result = model.fit()

# Print the results
print(result.summary())


                  Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       mmn      
No. Observations:       154           Method:                   REML     
No. Groups:             10            Scale:                    5.1360   
Min. group size:        12            Log-Likelihood:           -350.8831
Max. group size:        16            Converged:                Yes      
Mean group size:        15.4                                             
-------------------------------------------------------------------------
                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                       1.773    0.634  2.799 0.005  0.532  3.015
day[T.day2]                     1.274    0.514  2.477 0.013  0.266  2.282
day[T.day3]                    -0.455    0.507 -0.899 0.369 -1.449  0.538
day[T.day4]                     0.412    0.522  0.788 0.

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))


## Interpretation of the results
Looks like that the type of meditaiton does not have any significant effect on the mmn.  
When visually inspecting the MMNs this is confirmed, so yeah, looks like not much we can do. 

## Legacy Function: Old Old --> visualisation with all four tones

In [9]:
plot_all = True
for sub in range(2):
    for condition in ['jhana', 'mindfulness']:
        
        # Read epochs
        path = f"epochs/sub{sub}_{condition}_epo.fif"
        epochs = mne.read_epochs(path)
        evokeds = get_evokeds(epochs)
        
        # Compute mismatch data
        oddball = epochs["-1"].average()
        standard = epochs["1"].average()
        mismatch_data = standard.data - oddball.data
        mismatch = mne.EvokedArray(mismatch_data, standard.info, tmin=standard.times[0])

        if plot_all: 
            # Plot both evoked responses and mismatch negativity in one plot
            fig, ax = plt.subplots()
            mne.viz.plot_compare_evokeds(evokeds, ci=0.95, title=f'Global oddball evoked response and MMN: {condition}_sub_{sub}', axes=ax)
            mne.viz.plot_compare_evokeds({'MMN': mismatch}, ci=0.95, axes=ax, colors={'MMN': 'cyan'})
    
            
            # Set the y-axis limits
            ax.set_ylim([-10, 10])
            
            # Add vertical lines and shaded region
            ax.axvline(x=0.15, color='b', linestyle=':', label='Stimulus Onset 0.15s')
            ax.axvline(x=0.3, color='b', linestyle=':', label='Stimulus Onset 0.3s')
            ax.axvline(x=0.45, color='b', linestyle=':', label='Stimulus Onset 0.45s')
            ax.axvline(x=0.6, color='r', linestyle='--', label='Stimulus Onset 0.6s')
            ax.axvspan(0.75, 0.85, color='gray', alpha=0.3, label='MMN')
            ax.axvspan(0.85, 1.1, color='black', alpha=0.3, label='P3')
            ax.legend()
            
            # Save the figure
            fig.savefig(f'figures/sub{sub}_{condition}_combined.png')
            plt.close(fig)
        
        if condition == 'jhana':
            mismatch_jhana = mismatch.copy()
        if condition == 'mindfulness':
            mismatch_mindfulness = mismatch.copy()

    title = f'Mismatch Negativity (MMN) Response: sub_{sub}.png'
    fig, ax = plt.subplots()
    mne.viz.plot_compare_evokeds({'MMN_jhana': mismatch_jhana, 'MMN_mindfulness': mismatch_mindfulness}, ci=0.95, title=title, axes=ax)
    ax.set_ylim([-10, 10])
    ax.axvline(x=0.15, color='b', linestyle=':', label='Stimulus Onset 0.15s')
    ax.axvline(x=0.3, color='b', linestyle=':', label='Stimulus Onset 0.3s')
    ax.axvline(x=0.45, color='b', linestyle=':', label='Stimulus Onset 0.45s')
    ax.axvline(x=0.6, color='r', linestyle='--', label='Stimulus Onset 0.6s')
    ax.axvspan(0.75, 0.85, color='gray', alpha=0.3, label='MMN')
    ax.axvspan(0.85, 1.1, color='black', alpha=0.3, label='P3')
    ax.legend()
    fig.savefig(f'figures/{title}.png')
    plt.close(fig)

Reading /Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/MMN/epochs/2_by_subject/sub0_jhana_len4_epo.fif ...
    Found the data of interest:
        t =    -101.56 ...     800.78 ms
        0 CTF compensation matrices available
Not setting metadata
1443 matching events found
No baseline correction applied
0 projection items activated


NameError: name 'ax' is not defined